<a href="https://colab.research.google.com/github/natsakh/IAD/blob/main/Pr_6/6_6_Conv1D_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

import matplotlib.pyplot as plt
import numpy as np
import re, string
from collections import Counter

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import random
import torch.nn.functional as F

torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Device: cpu


In [2]:
# Hugging Face Datasets — бібліотека для завантаження датасетів.
#У Google Colab вона вже попередньо встановлена
# Якщо ви працюєте локально — попередньо виконайте:
#     pip install datasets
from datasets import load_dataset
#https://huggingface.co/datasets

In [3]:
ds = load_dataset("imdb")
#ds["train"], ds["test"]
#список словників — кожен елемент має ключі 'text' і 'label'

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [5]:
example = ds["train"][0]
print('text: ', example['text'])
print('label: ', example['label'])
#label = 1 → позитивний відгук, 0 → негативний.

text:  I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwe

In [6]:
def clean_text(text):
    text = text.lower()                                       # до нижнього регістру
    text = re.sub(r"@\S+", " ", text)                         # прибрати згадки @user
    text = re.sub(r"http\S+", " ", text)                      # прибрати посилання
    text = re.sub(r"[^a-z\s]", " ", text)                     # залишити лише букви (прибрати цифри, спецсимволи)
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)  # прибрати пунктуацію
    text = re.sub(r"\s+", " ", text)                          # замінити багато пробілів на один
    return text

In [7]:
cleaned_texts = [clean_text(ex["text"]) for ex in ds["train"]]
tokens = [word_tokenize(t) for t in cleaned_texts]

In [8]:
counter = Counter()
for tok_list in tokens:
    counter.update(tok_list)

In [9]:
# кількість унікальних слів
vocab_size = len(counter)

print(f"Кількість унікальних слів у train-наборі: {vocab_size:,}")

MAX_VOCAB = 20_000

Кількість унікальних слів у train-наборі: 73,206


In [10]:
# створюємо списки перетворень, створюємо словник
specials = ["<pad>", "<unk>"]
most_common = counter.most_common(MAX_VOCAB - len(specials))
itos = specials + [w for w, _ in most_common]       # index → string
stoi = {w: i for i, w in enumerate(itos)}           # string → index

PAD_IDX = stoi["<pad>"]
UNK_IDX = stoi["<unk>"]

print("Розмір словника:", len(stoi))
print("Приклад індексу:", stoi.get("movie", UNK_IDX))

Розмір словника: 20000
Приклад індексу: 19


In [11]:
#кодуємо тексти у послідовності індексів
#для кожного токена беремо його індекс зі словника stoi,
#якщо слова немає — повертаємо індекс UNK_IDX.
def encode(tokens):
    return [stoi.get(t, UNK_IDX) for t in tokens]

encoded_texts = [encode(tok_list) for tok_list in tokens]
print(encoded_texts[0][:20])


[11, 1595, 11, 240, 1973, 4046, 39, 61, 367, 1098, 87, 5, 32, 2, 6975, 13, 3357, 9, 55, 9]


In [12]:
#Padding (вирівнюємо довжину)
#Нейронна мережа очікує тензори однакової довжини,
#тому короткі тексти “доповнюємо” <pad>, а надто довгі — обрізаємо

MAX_LEN = 100   # довжина послідовності
def pad_sequence(seq):
    seq = seq[:MAX_LEN] + [PAD_IDX] * max(0, MAX_LEN - len(seq))
    return torch.tensor(seq, dtype=torch.long)

X = torch.stack([pad_sequence(seq) for seq in encoded_texts])
print(X.shape)   # [N, MAX_LEN]


torch.Size([25000, 100])


In [13]:
# мітки (labels)
y = torch.tensor([ex["label"] for ex in ds["train"]], dtype=torch.float32)
print(y.shape)   # [N]

torch.Size([25000])


In [14]:
#TensorDataset і DataLoader
dataset = TensorDataset(X, y)

# розділяємо на train/val
VAL_FRAC = 0.2
val_sz = int(len(dataset) * VAL_FRAC)
train_sz = len(dataset) - val_sz
train_ds, val_ds = random_split(dataset, [train_sz, val_sz],
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=128)


In [15]:
# використовуємо ті самі clean_text, токенайзер/word_tokenize, stoi, PAD_IDX, MAX_LEN

# Підготовка X_test, y_test
cleaned_test = [clean_text(ex["text"]) for ex in ds["test"]]
tokens_test  = [word_tokenize(t) for t in cleaned_test]
encoded_test = [[stoi.get(tok, UNK_IDX) for tok in toks] for toks in tokens_test]

def pad_sequence(seq):
    seq = seq[:MAX_LEN] + [PAD_IDX] * max(0, MAX_LEN - len(seq))
    return torch.tensor(seq, dtype=torch.long)

X_test = torch.stack([pad_sequence(seq) for seq in encoded_test])
y_test = torch.tensor([ex["label"] for ex in ds["test"]], dtype=torch.float32)

# 2) TensorDataset + DataLoader
test_ds = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)  # на тесті shuffle=False


In [16]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    train_loss = 0.0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(loader)

    return train_loss


In [17]:
def evaluate_one_epoch(model, loader, criterion, device):
    model.eval()
    valid_loss = 0.0

    with torch.no_grad():
      for x, y in loader:
          x, y = x.to(device), y.to(device)
          out = model(x)
          loss = criterion(out, y)
          valid_loss += loss.item()
    valid_loss /= len(loader)

    return valid_loss

In [18]:
#Convolutional Neural Networks for Sentence Classification - https://arxiv.org/abs/1408.5882
class TextCNN(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        emb_dim: int = 128,
        num_filters: int = 128,
        kernel_sizes=(3, 4, 5),
        num_classes: int = 1,        # 1 для бінарної класифікації (BCEWithLogitsLoss)
        pad_idx: int = 0,
        dropout: float = 0.5,
        use_pretrained_weight: torch.Tensor | None = None,
        freeze_emb: bool = False,
    ):
        super().__init__()
        # Embedding
        if use_pretrained_weight is None:
            self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        else:
            self.embedding = nn.Embedding.from_pretrained(
                use_pretrained_weight, freeze=freeze_emb, padding_idx=pad_idx
            )

        # Набір 1D-конволюцій з різними kernel sizes (in_channels=emb_dim, out_channels=num_filters)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=emb_dim, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):                 # x: [B, T] (індекси слів)
        emb = self.embedding(x)           # [B, T, E]
        emb = emb.transpose(1, 2)         # → [B, E, T] для Conv1d

        # Conv1d → ReLU → глобальний MaxPool по часовій осі
        conv_feats = []
        for conv in self.convs:
            # вихід: [B, C, T'] де T' = T - k + 1
            h = F.relu(conv(emb))
            # global max-over-time: [B, C]
            h = F.max_pool1d(h, kernel_size=h.size(2)).squeeze(2)
            conv_feats.append(h)

        z = torch.cat(conv_feats, dim=1)  # [B, C * len(K)]
        z = self.dropout(z)
        logits = self.fc(z)               # [B, num_classes]
        # Для BCEWithLogitsLoss повертаємо [B]; для CE — залишаємо [B, num_classes]
        return logits.squeeze(1) if logits.size(1) == 1 else logits


In [19]:
vocab_size = len(stoi)

model = TextCNN(
    vocab_size=len(stoi),
    emb_dim=128,
    num_filters=128,
    kernel_sizes=(3,4,5),
    num_classes=1,
    pad_idx=PAD_IDX,
    dropout=0.5,
)
print(model)


TextCNN(
  (embedding): Embedding(20000, 128, padding_idx=0)
  (convs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    (1): Conv1d(128, 128, kernel_size=(4,), stride=(1,))
    (2): Conv1d(128, 128, kernel_size=(5,), stride=(1,))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=384, out_features=1, bias=True)
)


In [20]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [21]:
# навчання
n_epochs = 7

for epoch in range(n_epochs):
   train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
   val_loss = evaluate_one_epoch(model, val_loader, criterion, device)

   print(f"[{epoch+1:02d}] train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")

[01] train_loss=0.6750 | val_loss=0.5464
[02] train_loss=0.5385 | val_loss=0.4936
[03] train_loss=0.4767 | val_loss=0.4635
[04] train_loss=0.4170 | val_loss=0.4362
[05] train_loss=0.3634 | val_loss=0.4112
[06] train_loss=0.3113 | val_loss=0.4181
[07] train_loss=0.2632 | val_loss=0.4086


In [22]:
# Для фінальної перевірки якості
@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_acc, n = 0.0, 0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = criterion(logits, yb)
        preds = (torch.sigmoid(logits) >= 0.5).long()
        total_loss += loss.item() * xb.size(0)
        total_acc  += (preds == yb.long()).sum().item()
        n += xb.size(0)
    return total_loss/n, total_acc/n

In [23]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"TEST  loss={test_loss:.4f}  acc={test_acc*100:.2f}%")

TEST  loss=0.4315  acc=80.48%
